In [118]:
import numpy as np
import pandas as pd

In [120]:
df=pd.read_csv("covid_toy.csv")

In [122]:
df.head(3)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No


In [124]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [126]:
from sklearn.impute import SimpleImputer
si=SimpleImputer()
df["fever"]=si.fit_transform(df[["fever"]])

In [128]:
df=pd.get_dummies(df,columns=["gender","city"])

In [130]:
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()
df["cough"]=oe.fit_transform(df[["cough"]])

In [132]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["has_covid"]=le.fit_transform(df[["has_covid"]])

C:\Users\shaba\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [134]:
df.isnull().sum()

age               0
fever             0
cough             0
has_covid         0
gender_Female     0
gender_Male       0
city_Bangalore    0
city_Delhi        0
city_Kolkata      0
city_Mumbai       0
dtype: int64

In [136]:
df.head(2)

,age,fever,cough,has_covid,gender_Female,gender_Male,city_Bangalore,city_Delhi,city_Kolkata,city_Mumbai
0,60,103.0,0.0,0,False,True,False,False,True,False
1,27,100.0,0.0,1,False,True,False,True,False,False


In [138]:
X=df.drop(columns="has_covid")
y=df["has_covid"]

In [140]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [146]:
df["has_covid"] = df["has_covid"].map({"positive": 1, "negative": 0})


In [148]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score


In [150]:

# Initialize a list to store the base learners
base_learners = []

# Number of base Learners (decision trees)
num_base_learners = 10

#Train the base learners

for i in range(num_base_learners):
    # Create a bootstrap sample of the training data
    bootstrap_indices = np.random.choice(len(X_train),
                                         size=len(X_train),replace=True)
    X_bootstrap = X_train.iloc[bootstrap_indices]
    y_bootstrap = y_train.iloc[bootstrap_indices]

    
    # Create and train a base learner (Random Forest)
    base_learner = RandomForestClassifier(n_estimators=10, random_state=42)
    base_learner.fit(X_bootstrap, y_bootstrap)

    #Add the trained base learner to the list
    base_learners.append(base_learner)

#Make predictions with each base learner
base_predictions=[]
for base_learner in base_learners:
    y_pred=base_learner.predict(X_test)
    base_predictions.append(y_pred)

#Combine the predictions using majority voting
ensemble_predictions = np.round(np.mean(np.array(base_predictions), axis=0))

accuracy=accuracy_score(y_test,ensemble_predictions)
print("Ensemble Accuracy:", accuracy)

Ensemble Accuracy: 0.5
